In [2]:
# coding:utf-8

from skimage import io,transform    
import glob 
import os   
import tensorflow as tf 
import numpy as np
import time

path='flower_photos/'  # 数据存放路径 
model_path='Model/model.ckpt'  # 模型保存路径
# 从原始数据集的每个类别中各自随机抽取一张图像进行模型验证
path1 = "flower_photos/daisy/19813618946_93818db7aa_m.jpg"
path2 = "flower_photos/dandelion/15987457_49dc11bf4b.jpg"
path3 = "flower_photos/roses/2677417735_a697052d2d_n.jpg"
path4 = "flower_photos/sunflowers/164670455_29d8e02bbd_n.jpg"
path5 = "flower_photos/tulips/3991423020_9aaf2b5974_n.jpg" 

# 定义花类字典，对每种花都赋值一个数值类别
flower_dict = {0:'dasiy',1:'dandelion',2:'roses',3:'sunflowers',4:'tulips'} 

# 定义转换之后测试花类图像的大小（长宽高分别是100,100,3）
w=100
h=100
c=3

# 定义read_one_image函数，用于将验证图像转换成统一大小的格式（100*100*3）
def read_one_image(path):               # 定义函数read_one_image
    img = io.imread(path)               # 利用io.imread函数读取图像 
    img = transform.resize(img,(w,h))   # 利用transform.resize函数对读取的图像数据进行格式统一化处理 
    return np.asarray(img)              # 对img图像数据进行转化 

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:              # 创建会话，用于执行已经定义的运算
    data = []                           # 定义空白列表，用于保存处理后的验证数据 
    data1 = read_one_image(path1)       # 利用自定义函数read_one_image依次对5张验证图像进行格式标准化处理
    data2 = read_one_image(path2)
    data3 = read_one_image(path3)
    data4 = read_one_image(path4)
    data5 = read_one_image(path5)
    data.append(data1)                  # 将处理过后的验证图像数据保存在前面创建的空白data列表当中
    data.append(data2)
    data.append(data3)    
    data.append(data4)
    data.append(data5)

    saver = tf.train.import_meta_graph('Model/model.ckpt.meta')     # 利用import_meta_graph函数直接加载之前已经持久化了的模型内容
    saver.restore(sess,tf.train.latest_checkpoint('Model'))         # 利用restore函数加载已经训练好的模型，并利用tf.train.latest_checkpoint函数提取最近一次保存的模型

    graph = tf.get_default_graph()              # 获取当前的默认计算图 

    x = graph.get_tensor_by_name("x:0")         # 返回给定名称的tensor
    # print(x)                                  # 返回加载的模型的参数 

    feed_dict = {x:data}                        # 利用feed_dict，给占位符传输数据 

    logits = graph.get_tensor_by_name("logits_eval:0")      # 返回logits_eval对应的tensor
    print(logits)

    classification_result = sess.run(logits,feed_dict)      # 利用feed_dict把数据传输到logits进行验证图像预测

    print(classification_result)                            # 打印预测矩阵
    print(tf.argmax(classification_result,1).eval())        # 打印预测矩阵每一行的最大值的下标 

    output = []                                             # 定义空白列表output
    output = tf.argmax(classification_result,1).eval()      # 选择出预测矩阵每一行最大值的下标，并将字符串str当成有效的表达式来求值并返回计算结果，将其赋值给output
    print(output)
    print(output.shape)

    for i in range(len(output)):                            # 遍历len(output)=5的花的类型
        print("flower",i+1,"prediction:"+flower_dict[output[i]])    # 输出每种花预测值最高的选项

D:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
D:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


INFO:tensorflow:Restoring parameters from Model\model.ckpt
Tensor("logits_eval:0", shape=(?, 5), dtype=float32)
[[ 13.567807   -3.6067247   2.9480975  -7.6485505  -6.658281 ]
 [ -6.471053   11.616253   -3.6437724  -7.403587   -2.1361136]
 [ -6.895006   -8.251003   19.64625   -13.329626    9.398162 ]
 [ -4.5452924   0.6525191  -3.325743    7.955768   -7.8973804]
 [-11.246075  -16.601582   13.731161   -9.659376   25.85981  ]]
[0 1 2 3 4]
[0 1 2 3 4]
(5,)
flower 1 prediction:dasiy
flower 2 prediction:dandelion
flower 3 prediction:roses
flower 4 prediction:sunflowers
flower 5 prediction:tulips
